In [ ]:
%%capture --no-stderr
%pip install -U langchain_openai langgraph langchain_core langchain_community pytrends requests beautifulsoup4 selenium webdriver_manager chromedriver-autoinstaller scikit-learn

In [271]:
import os, getpass


def _set_env(var: str):
    # Check if the variable is set in the OS environment
    env_value = os.environ.get(var)
    if not env_value:
        # If not set, prompt the user for input
        env_value = getpass.getpass(f"{var}: ")

    # Set the environment variable for the current process
    os.environ[var] = env_value


_set_env("OPENAI_API_KEY")

_set_env("SERP_API_KEY")

SERP_API_KEY = os.getenv("SERP_API_KEY")

In [272]:
context = """
            🚀 Contexto
            Hoje, o WhatsApp é um dos principais canais de vendas para pequenas e médias empresas (PMEs). No entanto, muitas delas enfrentam dificuldades em gerenciar mensagens, responder clientes a tempo e transformar conversas em vendas. O atendimento manual consome tempo e, muitas vezes, resulta em oportunidades perdidas.

                ❌ O Problema das PMEs
                📉 Perda de vendas por demora nas respostas ou atendimentos fora do horário.
                ⏳ Falta de tempo para responder manualmente a cada cliente.
                💬 Dificuldade em manter leads engajados e conduzi-los até a compra.
                🔗 Falta de integração com ferramentas que otimizam o processo de vendas.

            ✅ Nossa Solução: O Vendedor IA
            Criamos um Vendedor IA para WhatsApp, que automatiza o atendimento, responde clientes 24/7 e impulsiona as vendas sem necessidade de intervenção manual. Ele aprende sobre a marca, os produtos e os valores da empresa para oferecer um atendimento mais personalizado e eficiente.

                🎯 Principais Benefícios
                🤖 Atendimento Automático 24/7 – Nunca mais perca vendas por falta de resposta.
                ⚡ Respostas rápidas e inteligentes – Mantém clientes engajados no momento certo.
                🛠 Fácil de configurar – Sem necessidade de programação, pronto em minutos.
                📊 Dashboard com métricas – Acompanhe interações, leads e conversões.
                🔗 Integração com CRM – Gerencie leads e otimize o processo de vendas.
                📲 Totalmente conectado ao WhatsApp Business – Funcionamento fluido no principal canal de vendas das PMEs.

            🤖 Mais que um Chatbot: Seu Vendedor IA
                Muita gente já ouviu falar em chatbots, mas nosso Vendedor IA vai além!

                Os chatbots tradicionais apenas respondem perguntas de forma automática. Já o Vendedor IA foi criado para entender sua marca, aprender sobre seus produtos e vender de verdade no WhatsApp.

                O que isso significa na prática?
                🚀 Ele não só responde – ele engaja e converte.
                🧠 Aprende sobre sua empresa para representar seu negócio de forma personalizada.
                📊 Fornece métricas e insights, ajudando você a vender mais.
                🔗 Integra-se com WhatsApp Business e CRM para um fluxo de vendas completo.

                💡 Enquanto um chatbot apenas atende, nosso Vendedor IA trabalha ativamente para aumentar suas vendas!
        """

In [273]:
from typing import List, TypedDict
from pydantic import BaseModel, Field


class GeneratedWords(BaseModel):
    words: List[str] = Field(
        ..., description="List of the words beloging to a specific context"
    )


class GeneratedAutoCompleteSentences(BaseModel):
    sentences: List[str] = Field(
        ...,
        description="List of the autocomplete sentences beloging to a specific context",
    )


class QuestionsWithAutoComplete(BaseModel):
    question: str
    autocomplete_questions: list[str]


class QuestionsWithPAA(BaseModel):
    question: str
    related_questions: list[str]


class WordPopularity(TypedDict):
    word: str
    popularity: float


def generate_most_insteresting_context_words(context: str) -> list[str]:
    pass


def get_google_trends_word_popularity(words: list[str]) -> list[WordPopularity]:
    pass


def generate_autocomplete_questions(
    words_w_popularity: list[WordPopularity],
) -> list[QuestionsWithAutoComplete]:
    pass


def get_related_questions(questions: list[str]) -> list[QuestionsWithPAA]:
    pass

In [278]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI


def generate_most_insteresting_context_words(
    context: str, number_of_words: int = 10
) -> list[str]:

    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.0, max_tokens=12000).with_retry(
        stop_after_attempt=10
    )

    # Using with_structured_output feature to parse the output
    # to the pydantic models previous defined
    # with_retry -> https://python.langchain.com/api_reference/core/runnables/langchain_core.runnables.base.Runnable.html#langchain_core.runnables.base.Runnable.with_retry
    llm_w_structured_output = (
        ChatOpenAI(model="gpt-4o-mini", temperature=0.0)
        .with_structured_output(GeneratedWords)
        .with_retry(stop_after_attempt=10)
    )

    prompt = ChatPromptTemplate(
        [
            (
                "system",
                """Atue como um especialista em SEO e análise de tendências de busca. Gere uma lista de **{number_of_words} palavras-chave (palavras únicas)** relacionadas ao seguinte contexto:

                    ------------------------------------------------------------------
                    {context}
                    ------------------------------------------------------------------

                    🔹 **Requisitos:**  
                    1. As palavras devem ser altamente relevantes para pesquisas no Google.  
                    2. Inclua tanto **termos amplos** quanto **termos de cauda longa** para melhor identificação de tendências.  
                    3. Priorize palavras com potencial para aparecer em **Featured Snippets** e influenciar a seção **People Also Ask (PAA)**.  
                    4. Não inclua perguntas, apenas **termos principais** que possam ser usados para otimização de conteúdo.  
    
                """,
            ),
        ]
    )

    chain = prompt | llm_w_structured_output

    return chain.invoke(input={"context": context, "number_of_words": number_of_words})


relevant_words = generate_most_insteresting_context_words(context=context, number_of_words=50)

In [284]:
print(relevant_words.words)

['WhatsApp', 'vendas', 'atendimento', 'automação', 'PMEs', 'chatbot', 'inteligência', 'respostas', 'clientes', 'leads', 'engajamento', 'integração', 'CRM', 'WhatsApp Business', 'métricas', 'conversões', 'personalização', 'fluxo de vendas', 'suporte', '24/7', 'oportunidades', 'eficiência', 'configuração', 'fácil', 'impulsionar', 'negócios', 'conversas', 'atendimento automático', 'respostas rápidas', 'análise', 'insights', 'estratégia', 'marketing', 'ferramentas', 'otimização', 'experiência do cliente', 'conversão', 'interações', 'sucesso', 'crescimento', 'tecnologia', 'solução', 'desempenho', 'feedback', 'satisfação', 'relacionamento', 'comunicação']


In [283]:
import os
import json
from datetime import datetime
from typing import Generator
import requests


BKP_GOOGLE_TRENDS_REQUESTS_PATH: str = "./google_data/trends"
WORDS_POPULARITY_PATH: str = "./google_data/data/relevant_words.json"
WORDS_POPULARITY: dict[str, int]

with open(WORDS_POPULARITY_PATH, "r") as file:
    WORDS_POPULARITY = json.load(file)


def save_json(data: dict, folder_path: str, filename: str) -> str:
    """
    Salva um dicionário como um arquivo JSON em uma pasta específica.

    Args:
        data (dict): Dados a serem salvos em JSON.
        folder_path (str): Caminho da pasta onde o JSON será salvo.
        filename (str): Nome do arquivo JSON (ex: "dados.json").

    Returns:
        str: Caminho completo do arquivo salvo.
    """
    # Garante que a pasta exista
    os.makedirs(folder_path, exist_ok=True)

    # Caminho completo do arquivo
    file_path = os.path.join(folder_path, filename)

    # Escreve os dados no arquivo JSON
    with open(file_path, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4, ensure_ascii=False)

    return file_path


def save_json_with_timestamp(data: dict, folder_path: str, suffix: str) -> str:
    """
    Salva um dicionário como JSON, nomeando o arquivo com base na entrada e timestamp.

    Args:
        data (dict): Dados a serem salvos em JSON.
        folder_path (str): Caminho da pasta onde o JSON será salvo.
        prefix (str): Prefixo para o nome do arquivo (ex: "usuario").

    Returns:
        str: Caminho completo do arquivo salvo.
    """
    # Garante que a pasta exista
    os.makedirs(folder_path, exist_ok=True)

    # Obtém a data/hora atual formatada como YYYYMMDD_HHMMSS
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

    filename = f"{timestamp}_{suffix}.json"

    return save_json(data=data, folder_path=folder_path, filename=filename)


def google_trends_query(q: str) -> requests.Response:
    url: str = "https://serpapi.com/search?"
    params: dict = {
        "engine": "google_trends",
        "q": q,
        "data_type": "TIMESERIES",
        "hl": "pt-BR",
        "geo": "BR-SP",
        "date": "today 3-m",
        "api_key": SERP_API_KEY,
    }
    return requests.get(url=url, params=params)


def get_google_trends_word_popularity(
    words: list[str], step: int = 5
) -> dict[str, int]:

    indices: Generator[int, None, None] = range(0, len(words), step)

    list_words_popularity: list = []
    
    for k in indices:
        selected_words = words[k : k + step]
        if len(selected_words) == 1:
            if selected_words[0] in WORDS_POPULARITY:
                print(f"{selected_words[0]} is already in the data!")
                continue
            selected_words = selected_words * 2
        words_to_query = ",".join(selected_words)

        # with open("./google_trends_data/requests/20250311_211805_WhatsApp,atendimento,automação,vendas,chatbot.json", "r") as file:
        #     response = json.load(file)
        #     data = response
        # print(data)

        response = google_trends_query(q=words_to_query)
        data = response.json()
        save_json_with_timestamp(
            data=data,
            folder_path=BKP_GOOGLE_TRENDS_REQUESTS_PATH,
            suffix=words_to_query,
        )
        print(words_to_query)

        list_words_popularity.append(
            {
                query["query"]: query["value"]
                for query in data["interest_over_time"]["averages"]
            }
        )

    # add the words popularity to the dict
    [
        WORDS_POPULARITY.setdefault(key, value)
        for words_popularity in list_words_popularity
        for key, value in words_popularity.items()
    ]


get_google_trends_word_popularity(relevant_words.words, 1)

WhatsApp is already in the data!
vendas is already in the data!
atendimento is already in the data!
automação is already in the data!
PMEs is already in the data!
chatbot is already in the data!


inteligência,inteligência


KeyError: 'interest_over_time'

In [276]:
print(WORDS_POPULARITY)



{'WhatsApp': 71.22388059701493, 'atendimento': 72.94029850746269, 'automação': 61.92537313432836, 'vendas': 78.6268656716418, 'chatbot': 51.1044776119403, 'integração': 35.776119402985074, 'leads': 55.71641791044776, 'personalização': 29.52238805970149, 'CRM': 76.32835820895522, 'PMEs': 11.373134328358208, 'café': 81}


In [ ]:
with open(WORDS_POPULARITY_PATH, "w", encoding="utf-8") as f:
    json.dump(WORDS_POPULARITY, f, indent=4, ensure_ascii=False)

In [202]:
selected_words = {
    key: value
    for key, value in WORDS_POPULARITY.items()
    if key
    in [
        "WhatsApp",
        "atendimento",
        "automação",
        "vendas",
        "chatbot",
        "leads",
        "PMEs",
    ]
}


selected_words

{'WhatsApp': 71.22388059701493,
 'atendimento': 72.94029850746269,
 'automação': 61.92537313432836,
 'vendas': 78.6268656716418,
 'chatbot': 51.1044776119403,
 'leads': 55.71641791044776,
 'PMEs': 11.373134328358208}

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
import requests
import urllib.parse


def get_google_autocomplete_suggestions(query: str) -> list[str]:
    encoded_query = urllib.parse.quote(query)
    url = f"https://suggestqueries.google.com/complete/search?client=chrome&hl=pt-BR&gl=br&q={encoded_query}"
    # url = f"https://suggestqueries.google.com/complete/search?client=chromeq={encoded_query}"
    response = requests.get(url)

    if response.status_code == 200:
        return response.json()[
            1
        ]  # As sugestões estão na segunda posição da lista retornada
    return []


# def


def generate_autocomplete_questions(
    words_w_popularity: dict[str, int],
    context: str = context,
    number_of_sentences: int = 25,
) -> list[QuestionsWithAutoComplete]:

    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.0, max_tokens=12000).with_retry(
        stop_after_attempt=10
    )

    # Using with_structured_output feature to parse the output
    # to the pydantic models previous defined
    # with_retry -> https://python.langchain.com/api_reference/core/runnables/langchain_core.runnables.base.Runnable.html#langchain_core.runnables.base.Runnable.with_retry
    llm_w_structured_output = (
        ChatOpenAI(model="gpt-4o-mini", temperature=0.0)
        .with_structured_output(GeneratedAutoCompleteSentences)
        .with_retry(stop_after_attempt=10)
    )
    prompt = ChatPromptTemplate(
        [
            (
                "system",
                """
            Você é um especialista em SEO e padrões de pesquisa do Google. 
            Sua tarefa é gerar {number_of_sentences} fragmentos de pesquisa que iniciem frases comuns no Google, 
            sem completá-las totalmente, permitindo que o autocomplete preencha o restante.

            ⚠️ **Regras Importantes:**  
            - **Use apenas as palavras-chave listadas.** O contexto serve apenas para referência, **NÃO use palavras do contexto na saída.**  
            - **Priorize palavras-chave mais relevantes** (com maior peso na lista).  
            - **Produza frases INCOMPLETAS**, seguindo padrões comuns de pesquisa.  
            - **Evite frases genéricas ou sem clareza dentro do contexto.** Se uma frase gerada não fizer sentido como uma busca realista dentro do tema, descarte-a.  
            - **Se fizer sentido, combine duas palavras-chave para tornar a frase mais útil e natural.**  

            📌 **Contexto (Apenas para Referência, NÃO Usar Palavras na Saída):** {context}  

            📌 **Lista de Palavras-Chave e Relevância:**  
            (As palavras mais relevantes devem aparecer mais vezes.)  
            {words_w_popularity}  

            📌 **Padrões de Pesquisa a Seguir:**  
            - "Como [palavra-chave]..."  
            - "O que é [palavra-chave]..."  
            - "Vale a pena [palavra-chave]..."  
            - "Melhores [palavra-chave]..."  
            - "Dicas para [palavra-chave]..."  
            - "Exemplos de [palavra-chave]..."  

            ⚡ **Saída Esperada:**  
            - **As frases devem ser curtas e usar SOMENTE palavras da lista de palavras-chave.**  
            - **Frases genéricas ou que não fazem sentido dentro do contexto devem ser evitadas.**  
            - **Se necessário, combine duas palavras-chave para gerar buscas mais naturais e úteis.**  

            -------------------------------  
            **Exemplo de Entrada:**  

            **Contexto:** Vendas automáticas pelo WhatsApp para pequenas empresas  
            **Palavras-chave:**  
            - Automação de vendas: 100  
            - WhatsApp Business: 80  
            - CRM para PME: 60  
            - Chatbot: 50  
            - Geração de leads: 40  

            -------------------------------  
            **Saída Esperada:**  

            - "Como usar automação de vendas..."  
            - "O que é WhatsApp Business..."  
            - "Vale a pena CRM para PME..."  
            - "Melhores chatbots para WhatsApp..."  
            - "Dicas para geração de leads..."  
            - "Exemplos de automação no WhatsApp..."  

            🚨 **IMPORTANTE:**  
            - **As frases devem fazer sentido dentro do contexto!**  
            - **O Google completará a pesquisa via autocomplete.**  
            """,
            ),
        ]
    )
    # - **Não adicione palavras extras.**
    # - **As frases devem ser curtas e incompletas.**

    chain = prompt | llm_w_structured_output

    generated_sentences = chain.invoke(
        input={
            "context": context,
            "words_w_popularity": words_w_popularity,
            "number_of_sentences": number_of_sentences,
        }
    )

    # return generated_sentences

    return [
        QuestionsWithAutoComplete(
            question=sentence.replace("...", ""),
            autocomplete_questions=[sentence.replace("...", "")]
            + get_google_autocomplete_suggestions(sentence.replace("...", "")),
        )
        for sentence in generated_sentences.sentences
    ]


# relevant_words = generate_most_insteresting_context_words(context=context)

In [246]:
autocomplete_questions = generate_autocomplete_questions(
    words_w_popularity=selected_words, context=context, number_of_sentences=5
)
autocomplete_questions


# query = "Como automatizar o atendimento no WhatsApp"
# suggestions = get_google_autocomplete_suggestions(query)
# print(suggestions)

[QuestionsWithAutoComplete(question='Como melhorar o atendimento no WhatsApp', autocomplete_questions=['Como melhorar o atendimento no WhatsApp', 'como melhorar o atendimento pelo whatsapp', 'como melhorar o atendimento ao cliente via whatsapp', 'como melhorar o atendimento via whatsapp', 'como melhorar o whatsapp']),
 QuestionsWithAutoComplete(question='O que é automação de vendas', autocomplete_questions=['O que é automação de vendas', 'o que é automação da força de vendas', 'o que é automação de marketing', 'o que é automação comercial', 'o que é automação de sistemas', 'o que é automacao comercial', 'o que é automatização do atendimento']),
 QuestionsWithAutoComplete(question='Vale a pena usar chatbot para PMEs', autocomplete_questions=['Vale a pena usar chatbot para PMEs', 'vale a pena pagar o chat gpt', 'vale a pena assinar chat pt', 'vale a pena pagar chatgpt 4', 'vale a pena assinar o chatgpt']),
 QuestionsWithAutoComplete(question='Melhores práticas para geração de leads', aut

In [266]:
import os
from typing import Any, Dict

BKP_GOOGLE_SEARCH_REQUESTS_PATH = "./google_data/search/requests.json"


def save_json_response(file_path: str, q: str, response: requests.Response) -> None:

    if os.path.exists(file_path):
        try:
            with open(file_path, "r", encoding="utf-8") as f:
                data = json.load(f)
        except json.JSONDecodeError:
            data = {}
    else:
        data = {}

    try:
        response_data = response.json()
    except ValueError:
        response_data = {"error": "Resposta não é um JSON válido."}

    data[q] = response_data

    with open(file_path, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=4)


def get_google_related_questions(
    q: str, file_path: str = BKP_GOOGLE_SEARCH_REQUESTS_PATH
) -> Dict[str, Any]:
    url: str = "https://serpapi.com/search?"
    params: dict = {
        "q": q,
        "hl": "pt-BR",
        "geo": "BR-SP",
        "date": "today 3-m",
        "api_key": SERP_API_KEY,
    }

    query_result: Dict[str, Any] = None

    if file_path is not None and os.path.exists(file_path):
        try:
            with open(file_path, "r", encoding="utf-8") as f:
                data = json.load(f)
            if q in data:
                query_result = data[q]
        except (json.JSONDecodeError, IOError):
            pass
        
    if query_result is None:
        print(" to torando!!!!")
        response = requests.get(url=url, params=params)
        save_json_response(BKP_GOOGLE_SEARCH_REQUESTS_PATH, q, response)
        query_result = response.json()

    return [
        related_question["question"]
        for related_question in query_result["related_questions"]
    ]


def get_peolple_also_ask_questions(
    autocomplete_questions: list[QuestionsWithAutoComplete],
) -> list[QuestionsWithPAA]:

    return {
        autocomplete_question.question:

    
        {
            question: {
                'question': question,
                'related_questions': [question] + get_google_related_questions(q=question),
            }
            for question in autocomplete_question.autocomplete_questions
        }
        for autocomplete_question in autocomplete_questions
    }

In [267]:
set_of_questions_to_write_about =get_peolple_also_ask_questions(autocomplete_questions)


In [268]:
set_of_questions_to_write_about

{'Como melhorar o atendimento no WhatsApp': {'Como melhorar o atendimento no WhatsApp': {'question': 'Como melhorar o atendimento no WhatsApp',
   'related_questions': ['Como melhorar o atendimento no WhatsApp',
    'Como melhorar o atendimento ao cliente pelo WhatsApp?',
    'Como posso melhorar minha comunicação no WhatsApp?',
    'Como fazer um atendimento humanizado pelo WhatsApp?',
    'Qual o melhor CRM para WhatsApp?']},
  'como melhorar o atendimento pelo whatsapp': {'question': 'como melhorar o atendimento pelo whatsapp',
   'related_questions': ['como melhorar o atendimento pelo whatsapp',
    'Como melhorar o atendimento ao cliente no WhatsApp?',
    'Como posso melhorar minha comunicação no WhatsApp?',
    'Como fazer um atendimento humanizado pelo WhatsApp?',
    'Como atender um cliente por WhatsApp?']},
  'como melhorar o atendimento ao cliente via whatsapp': {'question': 'como melhorar o atendimento ao cliente via whatsapp',
   'related_questions': ['como melhorar o ate

In [269]:
set_of_questions_to_write_about =get_peolple_also_ask_questions(autocomplete_questions)


with open('./google_data/data/question_to_write_about.json', "w", encoding="utf-8") as f:
    json.dump(set_of_questions_to_write_about, f, ensure_ascii=False, indent=4)

In [297]:
set_of_questions_to_write_about

{'Como melhorar o atendimento no WhatsApp': {'Como melhorar o atendimento no WhatsApp': {'question': 'Como melhorar o atendimento no WhatsApp',
   'related_questions': ['Como melhorar o atendimento no WhatsApp',
    'Como melhorar o atendimento ao cliente pelo WhatsApp?',
    'Como posso melhorar minha comunicação no WhatsApp?',
    'Como fazer um atendimento humanizado pelo WhatsApp?',
    'Qual o melhor CRM para WhatsApp?']},
  'como melhorar o atendimento pelo whatsapp': {'question': 'como melhorar o atendimento pelo whatsapp',
   'related_questions': ['como melhorar o atendimento pelo whatsapp',
    'Como melhorar o atendimento ao cliente no WhatsApp?',
    'Como posso melhorar minha comunicação no WhatsApp?',
    'Como fazer um atendimento humanizado pelo WhatsApp?',
    'Como atender um cliente por WhatsApp?']},
  'como melhorar o atendimento ao cliente via whatsapp': {'question': 'como melhorar o atendimento ao cliente via whatsapp',
   'related_questions': ['como melhorar o ate

In [299]:
[autocomplete_questions[main_question] for main_question, autocomplete_questions in set_of_questions_to_write_about.items()]

[{'question': 'Como melhorar o atendimento no WhatsApp',
  'related_questions': ['Como melhorar o atendimento no WhatsApp',
   'Como melhorar o atendimento ao cliente pelo WhatsApp?',
   'Como posso melhorar minha comunicação no WhatsApp?',
   'Como fazer um atendimento humanizado pelo WhatsApp?',
   'Qual o melhor CRM para WhatsApp?']},
 {'question': 'O que é automação de vendas',
  'related_questions': ['O que é automação de vendas',
   'O que é automação e para que serve?',
   'Como posso automatizar o processo de vendas?',
   'Qual a função da automação comercial?',
   'Quais são as 5 técnicas de vendas?']},
 {'question': 'Vale a pena usar chatbot para PMEs',
  'related_questions': ['Vale a pena usar chatbot para PMEs',
   'Vale a pena investir em chatbots?',
   'Quais são as desvantagens do chatbot?',
   'Quais são as vantagens de usar chatbots?',
   'Quem precisa de um chatbot?']},
 {'question': 'Melhores práticas para geração de leads',
  'related_questions': ['Melhores práticas

In [303]:
import itertools

nested_list = [
    related_questions['related_questions'] 
    for main_question, autocomplete_questions in set_of_questions_to_write_about.items()
    for autocomplete_question, related_questions in autocomplete_questions.items()
]

flattened_related_questions = list(itertools.chain(*nested_list))
flattened_related_questions


['Como melhorar o atendimento no WhatsApp',
 'Como melhorar o atendimento ao cliente pelo WhatsApp?',
 'Como posso melhorar minha comunicação no WhatsApp?',
 'Como fazer um atendimento humanizado pelo WhatsApp?',
 'Qual o melhor CRM para WhatsApp?',
 'como melhorar o atendimento pelo whatsapp',
 'Como melhorar o atendimento ao cliente no WhatsApp?',
 'Como posso melhorar minha comunicação no WhatsApp?',
 'Como fazer um atendimento humanizado pelo WhatsApp?',
 'Como atender um cliente por WhatsApp?',
 'como melhorar o atendimento ao cliente via whatsapp',
 'Como melhorar o atendimento ao cliente pelo WhatsApp?',
 'Como atender um cliente por WhatsApp?',
 'Qual a melhor forma de abordar um cliente via WhatsApp?',
 'Como posso oferecer um atendimento humanizado via WhatsApp?',
 'como melhorar o atendimento via whatsapp',
 'Como melhorar o atendimento ao cliente no WhatsApp?',
 'Como posso melhorar minha comunicação no WhatsApp?',
 'Como fazer um atendimento humanizado pelo WhatsApp?',
 'C

In [304]:
import numpy as np
from sklearn.cluster import AgglomerativeClustering
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS

# 🔹 1️⃣ Gerar Embeddings para as Perguntas
embedding_model = OpenAIEmbeddings()
embeddings_matrix = np.array(embedding_model.embed_documents(flattened_related_questions))


In [306]:

# 🔹 2️⃣ Aplicar Clusterização Hierárquica
num_clusters = 8  # Ajustável conforme os dados
clustering_model = AgglomerativeClustering(n_clusters=num_clusters)
cluster_labels = clustering_model.fit_predict(embeddings_matrix)

# 🔹 3️⃣ Organizar perguntas por cluster
clusters = {i: [] for i in range(num_clusters)}
for i, question in enumerate(flattened_related_questions):
    clusters[cluster_labels[i]].append(question)

# 🔹 4️⃣ Gerar nomes automáticos para cada cluster (usando a primeira pergunta como referência)
print("\n📌 Estrutura do Artigo Gerada:\n")
for cluster, grouped_questions in clusters.items():
    title = grouped_questions[0]  # Podemos melhorar essa estratégia depois
    print(f"### {title}")
    for question in grouped_questions:
        print(f"- {question}")
    print()



📌 Estrutura do Artigo Gerada:

### O que é automação de vendas
- O que é automação de vendas
- O que é automação e para que serve?
- Como posso automatizar o processo de vendas?
- Qual a função da automação comercial?
- o que é automação da força de vendas
- O que é automação de força de vendas?
- O que significa automação de vendas?
- O que é um sistema de força de vendas?
- Qual a função da automação comercial?
- o que é automação de marketing
- Como fazer automação de marketing?
- Qual a melhor definição para automação de marketing?
- Quais são as ferramentas de automação de marketing?
- O que é automação de um exemplo?
- o que é automação comercial
- O que é um sistema de automação comercial?
- O que faz uma empresa de automação comercial?
- São exemplos de automação?
- O que é automação e para que serve?
- o que é automação de sistemas
- O que é um sistema de automação?
- O que se faz em automação?
- Quais são os 3 tipos de automação?
- São exemplos de automação?
- o que é automa